# Fragment Ion Intensity Prediction 

This notebook is prepared to be run in Google [Colaboratory](https://colab.research.google.com/). In order to train the model faster, please change the runtime of Colab to use Hardware Accelerator, either GPU or TPU.

This notebook presents a short walkthrough the process of reading a dataset and training a model for intensity prediction. The dataset is an example dataset extracted from a ProteomeTools dataset generated in the **Chair of Bioanalytics** at the **Technical University of Munich**.

DLOmix is the framework being used and is a custom wrapper on top of Keras/TensorFlow.

In [ ]:
# install the DLOmix package in the current environment using pip

!python -m pip install -q dlomix

The available modules in the framework are as follows:

In [ ]:
import numpy as np
import pandas as pd
import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports
print([x for x in dir(dlomix) if not x.startswith("_")])



**Note**: reports and pipelines are work-in-progress, some funtionalities are not complete.

- `constants`: constants to be used in the framework (e.g. Aminoacid alphabet mapping)
- `data`:  classes for representing dataset, wrappers around HuggingFace datasets to process input data and generate tensor datasets
- `eval`: custom evaluation metrics implemented in Keras/TF to work as `metrics` for model training
- `layers`: custom layer implementation required for the different models
- `models`: different model implementations for Retention Time and Fragment Ion Intensity Prediction
- `pipelines`: complete pipelines to run a task (e.g. Retention Time prediction)

**Note**: reports and pipelines are work-in-progress, some funtionalities are not complete.

## 1. Load Data

We can import the dataset class and create an object of type `FragmentIonIntensityDataset`. This object wraps around a Hugging Face dataset that can generate TensorFlow Dataset objects or Torch Dataset for training, validation, or testing. This can be controlled by the arguments `val_ratio`, `val_data_source`, and `test_data_source`.

In [ ]:
from dlomix.data import FragmentIonIntensityDataset

In [ ]:
TRAIN_DATAPATH = "https://github.com/wilhelm-lab/dlomix/raw/refs/heads/main/example_dataset/intensity/intensity_data.parquet"

int_data = FragmentIonIntensityDataset(
    data_format="parquet",
    data_source=TRAIN_DATAPATH,
    sequence_column="sequence",
    label_column="intensities",
    model_features=["precursor_charge_onehot", "collision_energy_aligned_normed"],
    max_seq_len=30,
    batch_size=128,
    val_ratio=0.2,
    with_termini=False,
)

Now we have an Intensity dataset that can be used directly with standard or custom `Keras` models. This wrapper contains the splits we chose when creating it. In our case, they are training and validation splits. To get the TF Dataset, we call the attributes `.tensor_rain_data` and `.tensor_val_data`.

In [ ]:
"Hugging Face Dataset", int_data

In [ ]:
 "Training examples", len(int_data["train"])

In [ ]:
 "Training examples", len(int_data["val"])

## 2. Model

We can now create the model. We will use a Prosit model [1]. It has the default working arguments, but most of the parameters can be customized.

**Note**: Important is to ensure that the padding length used for the dataset object is equal to the sequence length passed to the model.

*[1] Gessulat, S., Schmidt, T., Zolg, D. P., Samaras, P., Schnatbaum, K., Zerweck, J., ... & Wilhelm, M. (2019). Prosit: proteome-wide prediction of peptide tandem mass spectra by deep learning. Nature methods, 16(6), 509-518.*

In [ ]:
from dlomix.models import PrositIntensityPredictor

In [ ]:
model = PrositIntensityPredictor(
    seq_length=30,
    input_keys={
        "SEQUENCE_KEY": "sequence",
    },
    meta_data_keys={
        "COLLISION_ENERGY_KEY": "collision_energy_aligned_normed",
        "PRECURSOR_CHARGE_KEY": "precursor_charge_onehot",
    },
    with_termini=False,
)

## 3. Training

We can then train the model like a standard Keras model. The training parameters here are from Prosit, but other optimizer parameters can be used.  

In [ ]:
#imports

import tensorflow as tf
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance

In [ ]:
# create the optimizer object
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# compile the model  with the optimizer and the metrics we want to use, we can add our custom timedelta metric
model.compile(optimizer=optimizer,
              loss=masked_spectral_distance,
              metrics=['mse', masked_pearson_correlation_distance])

In [ ]:
history = model.fit(
    int_data.tensor_train_data,
    validation_data=int_data.tensor_val_data,
    epochs=10,
)

We store the result of training so that we can explore the metrics and the losses later. We specify the number of epochs for training and pass the training and validation data as previously described.

## 3. Testing and Reporting

We can create a test dataset to test our model. Additionally, we can use the reporting module to produce plots and evaluate the model.

Note: the reporting module is still in progress and some functionalities might easily break.

In [ ]:
# create the dataset object for test data
# Note: this is just using the same data above for demonstration purposes. In practice, you should use a separate test dataset

TEST_DATAPATH = "https://github.com/wilhelm-lab/dlomix/raw/refs/heads/main/example_dataset/intensity/intensity_data.parquet"

test_int_data = FragmentIonIntensityDataset(
    data_format="parquet",
    test_data_source=TEST_DATAPATH,
    sequence_column="sequence",
    label_column="intensities",
    model_features=["precursor_charge_onehot", "collision_energy_aligned_normed"],
    max_seq_len=30,
    batch_size=128,
    with_termini=False,
)

In [ ]:
test_int_data

In [ ]:
# Ensure columns in the test data are the same as in training data
int_data.column_names, test_int_data.column_names

In [ ]:
# use model.predict from keras directly on the testdata

predictions = model.predict(test_int_data.tensor_test_data)

In [ ]:
from dlomix.reports import IntensityReport

# create a report object by passing the history object and plot different metrics
report = IntensityReport(output_path="./output", history=history)

In [ ]:
# you can generate a complete report for intensity by calling generate_report
# the function takes the test dataset object and the predictions as arguments

report.generate_report(test_int_data, predictions, split="test")

In [ ]:
# the results used for plotting are availabe under the attribute prediction_results
report.prediction_results